In [37]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
from pymongo import MongoClient
import pprint
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [58]:
#Create web browser (do not need to have driver install - Safari driver for mac 
#is preloaded on the OS. Just need to enable: /usr/bin/safaridriver --enable)
driver = webdriver.Safari()
#Go to desired URL
driver.get('https://www.bluenile.com/diamond-search')
sleep(3)

For some reason the element is considered invisible when interacting with selenium by clicking it. The below code works

In [ ]:
#expanding the menu for all filters
expanded_filters = driver.find_element_by_xpath('//*[@id="react-app"]/div/div/section/div/div[2]/div[3]/button[2]')
driver.execute_script("arguments[0].click()",expanded_filters)
sleep(1)

#toggling on all additional filters
filters = ['Round-filter-button-med-lrg','Princess-filter-button-med-lrg','Emerald-filter-button-med-lrg',
             'Asscher-filter-button-med-lrg','Cushion-filter-button-med-lrg','Marquise-filter-button-med-lrg',
             'Radiant-filter-button-med-lrg','Oval-filter-button-med-lrg','Pear-filter-button-med-lrg',
             'Heart-filter-button-med-lrg','polish-toggle-button','symmetry-toggle-button',
           'fluorescence-toggle-button','depth %-toggle-button','table %-toggle-button','l/w ratio-toggle-button']
for element_id in filters:
    feature = driver.find_element_by_id(element_id)
    if feature.is_selected() == False:
        driver.execute_script("arguments[0].click()",feature)
        sleep(1)
cutlet=driver.find_element_by_xpath('//*[@id="culet-filter-container-accordion"]/button')
driver.execute_script("arguments[0].click()",cutlet)

In [41]:
#change "items per page" to 24
#need to use absolute xpath (full xpath) where you do not need to give any
#attribute names in the xpath because id is dynamic - cannot use
items_per_page = Select(driver.find_element_by_xpath('/html/body/div[1]/main/div/div/div/section/div/div[3]/div[2]/div/select'))
items_per_page.select_by_value('24')

In [57]:
next_page = driver.find_element_by_xpath('//button[text()="Next "]')
driver.execute_script("arguments[0].click()",next_page)

In [173]:
#iterating through all pages
num_page = 11691
#inserting intial page
page_source = driver.page_source
diamonds.insert_one({'page 1': 'html':page_source})
#clicking/iterating through each page and saving raw html to MongoDB
for page in range(2,num_page+1):
    #need to click to the next page
    diamonds.insert_one({f'page {page}': 'html':page_source}


In [56]:
driver.execute_script("window.scrollTo(0, window.scrollY + 300)")

There is a lazy loader but it doesn't truly load all entries even if it shows "items per page" as "all" - need to change "items per page" to the other option which is "24" and then iterate through all pages and pull data from each page instead.

In [107]:
#Scroll down the page forcing lazy loading
# for i in range(110):
#     driver.execute_script("window.scrollTo(0, window.scrollY + 600)")
    

In [117]:
#get page source - to use save into MongoDB and use for beautifulsoup
page_source = driver.page_source
client = MongoClient()
db = client.capstone2_bluenile
diamonds = db.diamond_info
#commenting out - do not want multiples if I rerun this notebook
#diamonds.insert_one({'website':'Blue Nile - Selenium','html':page_source})

In [124]:
#parse hypertext to get data with beautiful soup
soup = BeautifulSoup(page_source,'html.parser')

In [134]:
len(soup.find_all('div',class_="grid-row row"))

1000